In [2]:
# !pip install -r requirements.txt -i https://mirror.sjtu.edu.cn/pypi/web/simple

Looking in indexes: https://mirror.sjtu.edu.cn/pypi/web/simple
  Using cached https://mirror.sjtu.edu.cn/pypi-packages/30/03/4857f6a16c77bb090506d684d59aacc5ebf1722fa85748d7ea4d505b9ab3/langchain-0.0.146-py3-none-any.whl (600 kB)
  Using cached https://mirror.sjtu.edu.cn/pypi-packages/6d/9b/2f536f9e73390209e0b27b74691355dac494b7ec8154f3012fdc6debbae7/transformers-4.27.1-py3-none-any.whl (6.7 MB)
  Using cached https://mirror.sjtu.edu.cn/pypi-packages/94/0e/7ad36a6cffedea7d6dccb491d98ca4307d4a9f776d6d29d1045a82d453fd/unstructured-0.7.1-py3-none-any.whl (1.3 MB)
  Using cached https://mirror.sjtu.edu.cn/pypi-packages/08/cf/0bfbea1b2ace91af45e15bdec885e05992dc9150907a8398b3d305eddfd2/layoutparser-0.3.4-py3-none-any.whl (19.2 MB)
  Using cached https://mirror.sjtu.edu.cn/pypi-packages/a6/0a/0d20d2c0f16be91b9fa32a77b76c60f9baf6eba419e5ef5deca17af9c582/nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached https://mirror.sjtu.edu.cn/pypi-packages/20/9c/f07bd70d128fdb107bc02a0c702b9058b4fe147d0b

In [4]:
# pwd

'/root/autodl-tmp/langchain-ChatGLM'

In [2]:
# 引入包
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import UnstructuredFileLoader, DirectoryLoader
from models.chatglm_llm import ChatGLM # local file
import sentence_transformers
import torch
import os
import readline

In [3]:
# 全局参数
EMBEDDING_MODEL = "text2vec" # embedding 模型，对应 embedding_model_dict
VECTOR_SEARCH_TOP_K = 6
LLM_MODEL = "chatglm-6b"     # LLM 模型名，对应 llm_model_dict
LLM_HISTORY_LEN = 3
DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

embedding_model_dict = {
    "ernie-tiny": "nghuyong/ernie-3.0-nano-zh",
    "ernie-base": "nghuyong/ernie-3.0-base-zh",
    "text2vec": "/root/autodl-tmp/text2vec-large-chinese",
}

llm_model_dict = {
    "chatglm-6b-int4-qe": "THUDM/chatglm-6b-int4-qe",
    "chatglm-6b-int4": "THUDM/chatglm-6b-int4",
    "chatglm-6b": "/root/autodl-tmp/model/chatglm-6b", # /home/mw/input/ChatGLM6B6449
}


In [4]:
# 初始化配置
def init_cfg(LLM_MODEL, EMBEDDING_MODEL, LLM_HISTORY_LEN, V_SEARCH_TOP_K=6):
    global chatglm, embeddings, VECTOR_SEARCH_TOP_K
    VECTOR_SEARCH_TOP_K = V_SEARCH_TOP_K

    chatglm = ChatGLM()
    chatglm.load_model(model_name_or_path=llm_model_dict[LLM_MODEL])
    chatglm.history_len = LLM_HISTORY_LEN

    embeddings = HuggingFaceEmbeddings(model_name=embedding_model_dict[EMBEDDING_MODEL],)
    embeddings.client = sentence_transformers.SentenceTransformer(embeddings.model_name,
                                                                  device=DEVICE)

In [5]:
# 初始化指定知识库的 vector_store
def init_knowledge_vector_store(filepath:str):
    docs = []
    if not os.path.exists(filepath):
        print("路径不存在")
        return None
    elif os.path.isfile(filepath):
        file = os.path.split(filepath)[-1]
        try:
            loader = UnstructuredFileLoader(filepath, mode="elements")
            docs = loader.load()
            print(f"{file} 已成功加载")
        except:
            print(f"{file} 未能成功加载")
            return None
    elif os.path.isdir(filepath):
        try:
            loader = DirectoryLoader(filepath, glob="**/*.md")
            docs = loader.load()
            print(f"{filepath} 已成功加载")
        except Exception as e:
            print(f"{filepath} 未能成功加载: {e}")
            return None

    vector_store = FAISS.from_documents(docs, embeddings)
    return vector_store

In [6]:
# 结合知识库进行问题回答
def get_knowledge_based_answer(query, vector_store, chat_history=[]):
    global chatglm, embeddings

    prompt_template = """基于以下已知信息，简洁和专业的来回答用户的问题。
如果无法从中得到答案，请说 "根据已知信息无法回答该问题" 或 "没有提供足够的相关信息"，不允许在答案中添加编造成分，答案请使用中文。
已知内容:
{context}
问题:
{question}"""
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )
    chatglm.history = chat_history
    knowledge_chain = RetrievalQA.from_llm(
        llm=chatglm,
        retriever=vector_store.as_retriever(search_kwargs={"k": VECTOR_SEARCH_TOP_K}),
        prompt=prompt
    )
    knowledge_chain.combine_documents_chain.document_prompt = PromptTemplate(
            input_variables=["page_content"], template="{page_content}"
        )

    knowledge_chain.return_source_documents = True

    result = knowledge_chain({"query": query})
    chatglm.history[-1][0] = query
    return result, chatglm.history

In [7]:
# # 使用 Markdown 格式打印模型输出
# from IPython.display import display, Markdown, clear_output
# def display_answer(query, vector_store, history = []):
#     resp, history = get_knowledge_based_answer(query=query,
#                                                vector_store=vector_store,
#                                                chat_history=history)
#     display(Markdown(resp["result"]))
#     #return resp, history


In [8]:
# 执行初始化
init_cfg(LLM_MODEL, EMBEDDING_MODEL, LLM_HISTORY_LEN)
vector_store = init_knowledge_vector_store("/root/autodl-tmp/temp1_FAISS_20230612_110553")

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
2023-06-12 15:48:02.594860: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

No sentence-transformers model found with name /root/autodl-tmp/text2vec-large-chinese. Creating a new one with MEAN pooling.
No sentence-transformers model found with name /root/autodl-tmp/text2vec-large-chinese. Creating a new one with MEAN pooling.


/root/autodl-tmp/temp1_FAISS_20230612_110553 已成功加载


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_4862/2935205682.py:3 in <cell line: 3>                                            │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_4862/2935205682.py'                         │
│                                                                                                  │
│ /tmp/ipykernel_4862/3250167053.py:25 in init_knowledge_vector_store                              │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_4862/3250167053.py'                         │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/langchain/vectorstores/base.py:218 in               │
│ from_documents                                                                                   │
│                                                                                                  │
│   215 │   │   """Return VectorStore initialized from documents and embeddings."""                │
│   216 │   │   texts = [d.page_content for d in documents]                                        │
│   217 │   │   metadatas = [d.metadata for d in documents]                                        │
│ ❱ 218 │   │   return cls.from_texts(texts, embedding, metadatas=metadatas, **kwargs)             │
│   219 │                                                                                          │
│   220 │   @classmethod                                                                           │
│   221 │   async def afrom_documents(                                                             │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/langchain/vectorstores/faiss.py:368 in from_texts   │
│                                                                                                  │
│   365 │   │   │   │   faiss = FAISS.from_texts(texts, embeddings)                                │
│   366 │   │   """                                                                                │
│   367 │   │   embeddings = embedding.embed_documents(texts)                                      │
│ ❱ 368 │   │   return cls.__from(                                                                 │
│   369 │   │   │   texts,                                                                         │
│   370 │   │   │   embeddings,                                                                    │
│   371 │   │   │   embedding,                                                                     │
│                                                                                                  │
│ /root/miniconda3/lib/python3.8/site-packages/langchain/vectorstores/faiss.py:330 in __from       │
│                                                                                                  │
│   327 │   │   **kwargs: Any,                                                                     │
│   328 │   ) -> FAISS:                                                                            │
│   329 │   │   faiss = dependable_faiss_import()                                                  │
│ ❱ 330 │   │   index = faiss.IndexFlatL2(len(embeddings[0]))                                      │
│   331 │   │   index.add(np.array(embeddings, dtype=np.float32))                                  │
│   332 │   │   documents = []                                                                     │
│   333 │   │   for i, text in enumerate(texts):                                                   │
╰────────────────────────────────────────────────────────────

In [9]:
import faiss

index = faiss.read_index("/root/autodl-tmp/temp1_FAISS_20230612_110553/index.faiss")
print(index)


<faiss.swigfaiss_avx2.IndexFlat; proxy of <Swig Object of type 'faiss::IndexFlat *' at 0x7f1d70b7c480> >


In [11]:
import pickle

with open("/root/autodl-tmp/temp1_FAISS_20230612_110553/index.pkl", "rb") as f:
    obj = pickle.load(f)
    print(obj)


(<langchain.docstore.in_memory.InMemoryDocstore object at 0x7f1e60725790>, {0: '5c6d41da-914e-4969-b85f-bdc722286892', 1: 'bfb263d0-c05e-4a75-b657-a4ca3056671b', 2: '1af378e8-2b1e-4817-b25b-e90d05ffa8b7'})


# 测试

In [8]:
display_answer(query="ModelWhale 是什么",
                           vector_store=vector_store)

The dtype of attention mask (torch.int64) is not bool


ModelWhale 是一个在线数据科学平台，提供多种数据集和计算资源，支持用户进行数据分析和深度学习。用户可以在 ModelWhale 上创建自己的数据集、编写 Notebook 进行数据分析、运行深度学习模型等。ModelWhale 支持多种编程语言，包括 Python、R、Julia 等，并且提供了丰富的工具和功能，帮助用户进行数据预处理、可视化、模型训练和评估等。同时，ModelWhale 还提供了一些高级功能，如自定义计算引擎、计算资源的管理和共享等，方便用户进行高效的数据科学计算。

In [10]:
display_answer(query="ModelWhale 专业版与基础版的区别是什么？",
                           vector_store=vector_store)

ModelWhale 专业版和基础版是两种不同的版本，主要区别在于以下几个方面：

1. 价格：基础版的价格为 199 美元，而专业版的价格为 299 美元。

2. 功能：基础版只提供了一些基本的数据分析工具，如数据可视化、数据探索和数据建模等，而专业版则提供了更多的高级功能，如深度学习模型训练、自然语言处理、机器学习算法优化等。

3. 资源：专业版需要使用一些特殊的计算资源，如 GPU、TPU 等，这些资源通常比较昂贵，需要付费购买。基础版则可以使用普通的 CPU 和内存资源，不需要付费。

4. 用户权限：专业版用户可以获得更高的权限，可以访问更多的数据和算法，并且可以进行一些高级的优化和调整。基础版用户只能访问基础数据集和可视化工具，无法访问高级数据和算法。

5. 社区支持：专业版用户可以获得更好的社区支持和资源，可以与其他用户交流和合作，而基础版用户则没有这种优势。

总的来说，如果你是一名初学者或者需要一些简单的数据分析工具，那么基础版是一个不错的选择。如果你需要更高级的工具和权限，或者需要进行更深入的数据分析和研究，那么专业版可能更适合你。当然，如果你有足够的预算，或者愿意花费一些额外的费用，那么专业版也可以提供一些非常有价值的功能。

In [11]:
display_answer(query="为什么我的项目没有获得创作者收益",
                           vector_store=vector_store)

可能是因为该项目不符合“鲸选计划”的创作者要求。

“鲸选计划”旨在鼓励优秀的创作者在社区中分享他们的作品，提高社区的质量和内容数量。创作者需要满足以下要求才能参与“鲸选计划”:

1. 项目必须开源，并且包含高质量的内容。
2. 项目必须有一定的影响力，能够吸引一定数量的用户参与。
3. 项目必须遵守社区规则，不违反相关法律法规。
4. 项目必须有一定的创新性和实用性，能够为用户提供有价值的内容。

如果该项目不符合以上要求，那么它可能无法获得创作者收益。建议检查该项目是否符合“鲸选计划”的要求，如果不符合，可以尝试申请加入“鲸选计划”的团队，向管理员提交申请，以便获得更好的支持和管理。